In [38]:
import time
import numpy as np
from numpy import *
import matplotlib.pylab as plt
import operator
import pyscf
from pyscf import gto, scf, dft, tddft, data
import argparse

In [40]:
parser = argparse.ArgumentParser(description='Davidson')

parser.add_argument('-c', '--filename',       type=str, default='methanol.xyz', help='input filename')
parser.add_argument('-m', '--method',         type=str, default='RKS', help='RHF RKS UHF UKS')
parser.add_argument('-f', '--functional',     type=str, default='b3lyp', help='xc functional')
parser.add_argument('-b', '--basis_set',      type=str, default='def2-SVP', help='basis sets')
parser.add_argument('-i', '--initial_guess',  type=str, default='diag_A', help='initial_guess: diag_A or sTDA_A')
parser.add_argument('-p', '--preconditioner', type=str, default='diag_A', help='preconditioner: diag_A or sTDA_A')
parser.add_argument('-t', '--tolerance',      type=float, default='1e-5', help='residual norm convergence threshold')
parser.add_argument('-n', '--nstates',        type=int, default='4', help='number of excited states')

args = parser.parse_args()
################################################
# read xyz file and delete its first two lines
f = open(args.filename)
coordinates = f.readlines()
del coordinates[:2]
################################################

print ('args.method =', args.method)
###########################################################################
# build geometry in PySCF
mol = gto.Mole()
print (2)
mol.build(atom = coordinates, basis = args.basis_set, symmetry = True)
print (3)
###########################################################################

args.method = RKS
2
3


In [3]:
#np.set_printoptions (linewidth=300)

elements = ['H' , 'He', 'Li', 'Be', 'B' , 'C' , 'N' , 'O' , 'F' , 'Ne',
    'Na', 'Mg', 'Al', 'Si', 'P' , 'S' , 'Cl', 'Ar', 'K' , 'Ca',
    'Sc', 'Ti', 'V' , 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y' , 'Zr',
    'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
    'Sb', 'Te', 'I' , 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
    'Lu', 'Hf', 'Ta', 'W' , 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
    'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th',
    'Pa', 'U' , 'Np', 'Pu'] 
hardness = [
0.47259288,
0.92203391,
0.17452888,
0.25700733,
0.33949086,
0.42195412,
0.50438193,
0.58691863,
0.66931351,
0.75191607,
0.17964105,
0.22157276,
0.26348578,
0.30539645,
0.34734014,
0.38924725,
0.43115670,
0.47308269,
0.17105469,
0.20276244,
0.21007322,
0.21739647,
0.22471039,
0.23201501,
0.23933969,
0.24665638,
0.25398255,
0.26128863,
0.26859476,
0.27592565,
0.30762999,
0.33931580,
0.37235985,
0.40273549,
0.43445776,
0.46611708,
0.15585079,
0.18649324,
0.19356210,
0.20063311,
0.20770522,
0.21477254,
0.22184614,
0.22891872,
0.23598621,
0.24305612,
0.25013018,
0.25719937,
0.28784780,
0.31848673,
0.34912431,
0.37976593,
0.41040808,
0.44105777,
0.05019332,
0.06762570,
0.08504445,
0.10247736,
0.11991105,
0.13732772,
0.15476297,
0.17218265,
0.18961288,
0.20704760,
0.22446752,
0.24189645,
0.25932503,
0.27676094,
0.29418231,
0.31159587,
0.32902274,
0.34592298,
0.36388048,
0.38130586,
0.39877476,
0.41614298,
0.43364510,
0.45104014,
0.46848986,
0.48584550,
0.12526730,
0.14268677,
0.16011615,
0.17755889,
0.19497557,
0.21240778,
0.07263525,
0.09422158,
0.09920295,
0.10418621,
0.14235633,
0.16394294,
0.18551941,
0.22370139]
#in Hartree
HARDNESS = dict(zip(elements,hardness))

In [4]:
# mol = gto.Mole()
# mol.build(atom = '\
# C         -4.89126        3.29770        0.00029;\
# O         -3.49307        3.28429       -0.00328;\
# H         -5.28213        2.58374        0.75736;\
# H         -5.28213        3.05494       -1.01161;\
# H         -5.23998        4.31540        0.27138;\
# H         -3.22959        2.35981       -0.24953',\
# basis = 'def2-SVP')


In [5]:
# mol = gto.Mole()
# mol.build(atom = '\
# C         -3.15617        2.59898        0.79547;\
# C         -1.79169        2.11570        0.42917;\
# O         -0.80893        2.56621        0.99508;\
# H         -1.66947        1.36193       -0.34183;\
# H         -3.35300        2.38970        1.86780;\
# H         -3.91803        2.07820        0.17854;\
# H         -3.22824        3.69190        0.61449',\
# basis = 'def2-SVP')

In [6]:
# mol = gto.Mole()
# mol.build(atom = 'O         -4.89126        3.29770        0.00029;\
# H         -3.49307        3.28429       -0.00328;\
# H         -5.28213        2.58374        0.75736', basis = 'def2-SVP', symmetry = True)

In [7]:
# mol = gto.Mole()
# mol.build(atom = '\
# C         -1.44673        2.80824       -0.07813;\
# O         -1.78998        3.80792       -0.69188;\
# N         -1.18291        1.66325       -0.74926;\
# N         -1.33221        2.85136        1.26946;\
# H         -1.53168        3.72026        1.78651;\
# H         -1.04365        2.01512        1.79805;\
# H         -1.26824        1.62382       -1.77554;\
# H         -0.89270        0.81388       -0.24298'\
# , basis = 'def2-SVP', symmetry = True)

In [8]:
# mf = dft.RKS(mol) 
# mf.conv_tol = 1e-12
# mf.grids.level = 9     # 0-9, big number for large mesh grids, default is 3
# mf.xc = 'b3lyp'
# mf.kernel()  #single point energy

In [9]:
# mf = dft.RKS(mol) 
# mf.conv_tol = 1e-12
# mf.grids.level = 9     # 0-9, big number for large mesh grids, default is 3
# mf.xc = 'cam-b3lyp'
# mf.kernel()  #single point energy

#ORCA: FINAL SINGLE POINT ENERGY = -115.576160742154
#Turbomole: total energy      =    -115.57615989622

In [10]:
mf = scf.RHF(mol) 
mf.conv_tol = 1e-13
mf.kernel()  #single point energy

converged SCF energy = -114.950987102225


-114.95098710222453

In [11]:
# CIS = pyscf.tdscf.rhf.CIS(mf)
# CIS.nstates = 5
# CIS.kernel()

In [12]:
# td = tddft.TDA(mf)
# start = time.time()
# td.kernel()    #compute first few excited states.
# end = time.time()
# print ('Pyscf time =', round(end-start,4))

In [13]:
#mf.analyze()
##MO energies

In [14]:
#mf.mulliken_pop_meta_lowdin_ao()
#population analysis
#mf.mulliken_pop()

In [15]:
#check whether a is symmetric
def check_symmetric(a, tol=1e-12):
    return np.all(np.abs(a-a.T) < tol)

In [16]:
def matrix_power (S,a):
    s,ket = np.linalg.eigh(S)
    # S = mf.get_ovlp() #.get_ovlp() is basis overlap matrix
    # S = np.dot(np.linalg.inv(c.T), np.linalg.inv(c))
    # #s are eigenvalues, must be all positive
    # #each column of ket is a eigenket
    s = s**a
    X = np.linalg.multi_dot([ket,np.diag(s),ket.T])
    #X == S^1/2
    return X

def orthonormalize (C):
    X = matrix_power(mf.get_ovlp(), 0.5)
    C = np.dot(X,C)
    return C
# C is orthonormalized coefficient matrix
# np.dot(C.T,C) is a identity matrix

def coefficient_matrix ():
    C = mf.mo_coeff
    # mf.mo_coeff is the coefficient matrix
    C = orthonormalize (C)
    return C
# rthogonalized MO coefficients 


In [17]:
def generateQ (atom_id):
    q = np.zeros([N_bf, N_bf])
    #N_bf is number Atomic orbitals, q is same size with C
    
    C = coefficient_matrix ()
    for i in range (0, N_bf):
        for p in range (0, N_bf):
            for mu in range (0, N_bf):
                if AO[mu] == atom_id:
                    #collect all basis functions centered on atom_id
                    # the last loop is to sum up all C_mui*C_mup, calculate element q[i,p]
                    q[i,p] += C[mu,i]*C[mu,p]
                    #q[i,p] += 2*C[i,mu]*C[p,mu]
    return q

#home_made population analysis
# for atom_id in range (0, Natm):
#     print (check_symmetric(Qmatrix[atom_id], tol=1e-12))
#     m = 0
#     for i in range (0, occupied):
#         m += Qmatrix[atom_id][i,i]
#         #sum over occupied orbitals
#     print (m)

In [18]:
#function to return chemical hardness from dictionary HARDNESS
def Hardness (atom_id):
    atom = mol.atom_pure_symbol(atom_id) 
    return HARDNESS[atom]
# mol.atom_pure_symbol(atom_id) returns pure element symbol, no special characters

In [19]:
def eta (atom_A_id, atom_B_id):
    eta = (Hardness(atom_A_id) + Hardness(atom_B_id))/2
    return eta

In [20]:
def gammaJ (atom_A_id, atom_B_id):
    gamma_A_B_J = (R[atom_A_id, atom_B_id]**beta + (a_x * eta(atom_A_id, atom_B_id))**(-beta))**(-1/beta)
    return gamma_A_B_J

def gammaK (atom_A_id, atom_B_id):
    gamma_A_B_K = (R[atom_A_id, atom_B_id]**alpha + eta(atom_A_id, atom_B_id)**(-alpha)) **(-1/alpha)
    return gamma_A_B_K

In [21]:
#define two electron intergeral (pq|rs)
def ele_intJ (i,j,a,b):
    Natm = mol.natm 
    #number of atoms
    ijab = 0
    for atom_A_id in range (0, Natm):
        for atom_B_id in range (0, Natm):
            ijab += Qmatrix[atom_A_id][i,j] * Qmatrix[atom_B_id][a,b] * GammaJ[atom_A_id, atom_B_id]
    return ijab
        
def ele_intK (i,a,j,b):
    Natm = mol.natm 
    iajb = 0
    for atom_A_id in range (0, Natm):
        for atom_B_id in range (0, Natm):
            iajb += Qmatrix[atom_A_id][i,a] * Qmatrix[atom_B_id][j,b] * GammaK[atom_A_id, atom_B_id]
    return iajb

In [22]:
Natm = mol.natm
MOe = mf.mo_energy  
#an array of MO energies, in Hartree

occupied = len(np.where(mf.mo_occ > 0)[0])
#mf.mo_occ is an array of occupance [2,2,2,2,2,0,0,0,0.....]
virtual = len(np.where(mf.mo_occ == 0)[0])


AO = [int(i.split(' ',1)[0]) for i in mol.ao_labels()] 
# .split(' ',1) is to split each element by space, split once.
# mol.ao_labels() it is Labels of AO basis functions, AO is a list of corresponding atom_id
# AO == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
N_bf = len(AO)

Qmatrix = [(generateQ(atom_id)) for atom_id in range (0, Natm)]
#a list of q matrix

R = pyscf.gto.mole.inter_distance(mol, coords=None) 
#Inter-particle distance array
# unit == ’Bohr’, Its value is 5.29177210903(80)×10^(−11) m

a_x = 0.38
beta1= 1.86
beta2=0
alpha1= 0.9
alpha2=0
beta = beta1 + beta2 * a_x
alpha = alpha1 + alpha2 * a_x


#build gammaJ and gammaK matrix
GammaJ = np.zeros([Natm, Natm])
for i in range (0, Natm):
    for j in range (0, Natm):
        GammaJ[i,j] = gammaJ (i,j)

GammaK = np.zeros([Natm, Natm])
for i in range (0, Natm):
    for j in range (0, Natm):
        GammaK[i,j] = gammaK (i,j)

In [23]:
#build A matrix
def build_sTDA_A ():
    
    A = np.zeros ([occupied*virtual, occupied*virtual])
    m = -1
    for i in range (0, occupied):
        for a in range (occupied, N_bf):
            m += 1 #for each ia pair, it corresponds to a certain row
            n = -1
            for j in range (0, occupied):
                for b in range (occupied, N_bf):
                    n += 1 #for each jb pair, it corresponds to a certain column        
                    if i==j and a==b:
                        A[m,n] = (MOe[a]-MOe[i]) + 2*ele_intK(i,a,j,b) - ele_intJ(i,j,a,b)
                    else:
                        A[m,n] = 2*ele_intK(i,a,j,b) - ele_intJ(i,j,a,b)
    print ('sTDA_A matrix built, size =', np.shape(A)[0])
    return A

# start = time.time()
# A = build_sTDA_A ()
# end = time.time()
# print ('A_sTDA building time =', round (end - start, 2))

In [24]:
# print (check_symmetric(A, tol=1e-8))

In [25]:
# plt.matshow(A)
# plt.show()

In [26]:
# eigv,eigk = np.linalg.eigh(A)
# idx = eigv.argsort()
# eigv = eigv[idx]    #eigenvalues
# eigk = eigk[:,idx]          #eigenkets, m*m

# #np.linalg.eigh guarantees you that the eigenvalues are sorted and uses a faster algorithm 
# #that takes advantage of the fact that the matrix is symmetric. 

# print (np.round (eigv[:10]*27.21138624598853,4))   
# # ':n', first n elements; 'n:' all elements except firt n 


#methanol
# a_x = 0.38
# beta1= 1.86
# beta2=0
# alpha1= 0.9
# alpha2=0
#[12.2388 13.5994 13.7915 15.0407 15.1073 15.245  16.4534 16.5181 16.8506 17.3015]

In [27]:
td = tddft.TDA(mf)
# td.nstates = 5
# td.conv_tol = 1e-13
# td.kernel()

In [28]:
# n = occupied * virtual
# I = np.eye(n)
# H = np.zeros((n,n))

# vind, hdiag = td.gen_vind(mf)
# for i in range (0, n):
#     H[:,i] = vind (I[:, i])

# print (type(type(H)))
# print (type(vind))

In [29]:
# energies, vectors = np.linalg.eigh(H)
# print (energies[:5]*27.21138624598853)

In [30]:
sTDA_A = build_sTDA_A()
## prepare sTDA_A matrix
vind, hdiag = td.gen_vind(mf)
n = len(hdiag)
I = np.eye(n)

sTDA_A matrix built, size = 351


In [31]:
######################################################################################
def A_diag_initial_guess (k):
    m = 2*k  
    # m is size of subspace Hamiltonian, amount of initial guesses   
    # m=k works for H2, m=4k works for H2O
    V = np.zeros((n, m)) 
    #array of zeros, a container to hold current guess vectors 
    W = np.zeros((n, m)) 
    
    sort = hdiag.argsort()
    for j in range(0,m):
        V[int(np.argwhere(sort == j)), j] = 1   
        # positions with lowest values set as 1
        W[:, j] = vind(V[:, j])
    # W = Av, create transformed guess vectors

    return (m, V, W)

def sTDA_initial_guess (k):
    m = 2*k  
    # m is size of subspace Hamiltonian, amount of initial guesses   
    # m=k works for H2, m=4k works for H2O
    V = np.zeros((n, m)) 
    # array of zeros, a container to hold current guess vectors, v
    W = np.zeros((n, m)) 
    # a container to hold transformed guess vectors, Av
    eigvalues, eigkets = np.linalg.eigh(sTDA_A)
    # eigv, eigk = Davidson (A, m, 1e-5, default_initial_guess, A_diag_preconditioner)
    #!!!!!!!! diagonalize sTDA_A amtrix
    for j in range(0,m):
        V[:, j] = eigkets [:, j]
        W[:, j] = vind(V[:, j])
        
    return (m, V, W)
######################################################################################



###################################################################
def A_diag_preconditioner (residual, sub_eigenvalue, x):
    d = hdiag - sub_eigenvalue[x]
    d[(d<1e-16)&(d>=0)] = 1e-16
    d[(d>-1e-16)&(d<0)] = -1e-16   
    #kick out all small values
    
    new_vec = residual/d
    return new_vec

def sTDA_preconditioner (residual, sub_eigenvalue, x):
    new_vec = np.dot(np.linalg.inv(sTDA_A - sub_eigenvalue[x]*I),residual)
    return new_vec
####################################################################






################################################################################
def Davidson (k, tol, initial_guess, preconditioner):

    if initial_guess == 'sTDA_initial_guess':
        initial_guess = sTDA_initial_guess
        print ('Initial guess: sTDA A matrix')
    elif initial_guess == 'A_diag_initial_guess':
        initial_guess = A_diag_initial_guess
        print ('Initial guess: Diagonal of full A matrix')
        
    if preconditioner == 'sTDA_preconditioner':
        preconditioner = sTDA_preconditioner
        print ('Preconditioner: sTDA A matrix')
    elif preconditioner == 'A_diag_preconditioner':
        preconditioner = A_diag_preconditioner
        print ('Preconditioner: Diagonal of full A matrix')
    start = time.time()
    
    #tol = 1e-5      
    # Convergence tolerance
    
    max = 90      
    # Maximum number of iterations

    ########################################################################################### 
    for i in range(0, max):
        
        sum_convec = 0
        # total converged eigenvectors
        # breaf if sum_convec == k
        
        #################################################
        # generate initial guess
        if i == 0:
            m, V, W = initial_guess(k)
        #################################################

        sub_A = np.dot(V.T, W)  
        # sub_A is subspace A matrix
        sub_eigenvalue, sub_eigenket = np.linalg.eigh(sub_A) 
        # Diagonalize the subspace Hamiltonian, and sorted.

        lasit_newvec = 0
        # amount of new vectors added in last iteration, ranging from 1 to k
        # because not all new guess_vectors can survive the Gram-Schmidt

        ####################################################################################
        for x in range(0,k):      
            #looking at first k vecrors one by one, check whether they are roots
            residual = np.dot((W[:,:m]- sub_eigenvalue[x]*V[:,:m]), sub_eigenket[:,x])
            # np.dotV([:,:m])s[:,x]) can project the subspace-eigenket back to full space 

            norm = np.linalg.norm(residual)
            if norm <= tol:
                sum_convec += 1
            else:
                # current guess is not good enough, 
                # so we use current guess to create new guess vectors
                #########################################################
                new_vec = preconditioner (residual, sub_eigenvalue, x)          
                #########################################################
                # preconditioner
                
                new_vec = new_vec/np.linalg.norm (new_vec) 
                # normalize before Gram-Schmidt 
                for y in range (0, m + lasit_newvec):  
                    # orthornormalize the new vector against all old vectors
                    new_vec = new_vec - np.dot(V[:,y], new_vec) * V[:,y]   

                norm = np.linalg.norm (new_vec)
                if norm > 1e-16:
                    new_vec = new_vec/norm
                    # normalzie the new vector, now Gram-Schmidt is done

                    V = np.append (V, new_vec[:, None], axis=1)
                    # put the new guess into container
                    
                    trans_new_vec = vind(new_vec)
                    # print ('Shape of trans_new_vec =', np.shape(trans_new_vec))
                    W = np.append (W, trans_new_vec.T, axis = 1)
                    # put transformed guess Av into container
                    lasit_newvec += 1
        ####################################################################################      
        if sum_convec == k:
            break
        m += lasit_newvec
    ########################################################################################### 

    Eigenkets = np.dot(V[:,:m], sub_eigenket[:, :k])

    end = time.time()
    print ('Iteration steps =', i+1)
    print ('Davidson time:', round(end-start,4))
    
    return (sub_eigenvalue[:k]*27.21138624598853, Eigenkets[:,:k])



print ('#################################################################')
print ('In-house Davdison codes:')






k = 4
tol = 1e-5
initial_guess = 'sTDA_initial_guess'
preconditioner = 'sTDA_preconditioner'

print ('Number of excited states =', k)
print ('Residual convergenve threshold =', tol)

Excitation_energies, kets = Davidson (k, tol, initial_guess, preconditioner)

print ('Excited State energies (eV) =')
print (Excitation_energies)
print ('#################################################################')
print ('PySCF TDA-TDDFT codes:')
td.nstates = k
td.kernel()
print ('#################################################################')

# H2O Excited State energies (eV)
# [ 5.12051837  7.78183026  8.43305887  9.81364248 10.41186311]
# [ 5.12051842  7.78183032  8.43305894  9.81364256 10.4118632 ]
#


#CH3OH Excited State energies (eV)
#[ 8.73907144 10.77713239 11.26755041 12.203639   12.27658757]
#[ 8.73907152 10.77713248 11.2675505  12.2036391  12.27658768]



# CH3OH
# (351, 351)
# sTDA_A matrix built
# sTDA A matrix as initial guess
# sTDA A matrix as preconditioner
# Iteration steps = 11
# Davidson time: 0.3732
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]

# (351, 351)
# sTDA_A matrix built
# sTDA A matrix as initial guess
# Diagonal of full A matrix as preconditioner
# Iteration steps = 15
# Davidson time: 0.3009
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]


# (351, 351)
# sTDA_A matrix built
# Diagonal of full A matrix as initial guess
# Diagonal of full A matrix as preconditioner
# Iteration steps = 15
# Davidson time: 0.3152
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]


# (351, 351)
# sTDA_A matrix built
# Diagonal of full A matrix as initial guess
# sTDA A matrix as preconditioner
# Iteration steps = 11
# Davidson time: 0.334
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]

#################################################################
In-house Davdison codes:
Number of excited states = 4
Residual convergenve threshold = 1e-05
Initial guess: sTDA A matrix
Preconditioner: sTDA A matrix
Iteration steps = 11
Davidson time: 0.388
Excited State energies (eV) =
[ 8.73907152 10.77713248 11.2675505  12.2036391 ]
#################################################################
PySCF TDA-TDDFT codes:
Excited State energies (eV)
[ 8.73907144 10.77713239 11.26755041 12.203639  ]
#################################################################


In [32]:
def davidson_A_matrix_hstack (A, k): # matrix A and how many eignvalues to solve
    
    start = time.time()

    tol = 1e-5      # Convergence tolerance
    max = 40      # Maximum number of iterations

    ###########################################################################################
    # Begin iterations
    for i in range(0, max):
        sum_convec = 0
        #total converged eigenvectors
        # if sum_convec == k, break
        
        lasit_newvec = 0
        # it records amount of new vectors added in last iteration, ranging from 1 to k
        # because not all new guess_vectors can survive the Gram-Schmidt
        

        #################################################
        # generate initial guess
        if i == 0:
            #initial guess  
            n = np.shape(A)[0]
            m = k  
            # m is size of subspace Hamiltonian, amount of initial guesses   
            # m=k works for H2, m=4k works for H2O

            V = np.zeros((n, m)) #array of zeros, a container to hold current guess vectors
            
            sort = np.diag(A).argsort()
            for j in range(0,m):
                V[int(np.argwhere(sort == j)), j] = 1   
                # positions with lowest values set as 1
            W = np.dot(A,V)   
            # W = Av, create transformed guess vectors
        #################################################
        
        
        sub_A = np.dot(V.T, W)  
        # sub_A is subspace A matrix
        sub_eigenvalue, sub_eigenket = np.linalg.eigh(sub_A) 
        # Diagonalize the subspace Hamiltonian, and sorted.
        
        
        ####################################################################################
        for x in range(0,k):      
            #looking at first k vecrors one by one, check if they are roots
            residual = np.dot((W[:,:m]- sub_eiegnvalue[x]*V[:,:m]), sub_eigenket[:,x])
            # np.dotV([:,:m])s[:,x]) can transform the subspace-eigenket back into full space eigenket
            
            norm = np.linalg.norm(residual)
            if norm <= tol:
                sum_convec += 1
            else:
                #print ('norm > tol')
                # current guess is not good enough, 
                # so we use current guess to create new guess vectors
                d = np.diag(A)-sub_eiegnvalue[x]
                d[(d<1e-16)&(d>=0)] = 1e-16
                d[(d>-1e-16)&(d<0)] = -1e-16   
                # kick out all small values
                new_vec = residual/d          
                # preconditioner
                
                new_vec = new_vec/np.linalg.norm (new_vec) 
                # normalize before Gram-Schmidt 

                for y in range (0, m + lasit_newvec):  
                    # orthornormalize the new vector against all old vectors
                    new_vec = new_vec - np.dot(V[:,y], new_vec) * V[:,y]   
                    
                norm = np.linalg.norm (new_vec)
                if norm > 1e-16:
                    new_vec = new_vec/norm
                    # normalzie the new vector, now Gram-Schmidt is done
                    
                    V = np.append (V, new_vec[:, None], axis=1)
                    # put the new guess into container
                    
                    W = np.append (W, np.dot(A, new_vec)[:, None], axis = 1)
                    # put transformed guess Av into container
                    
                    lasit_newvec += 1
        ####################################################################################      
        if sum_convec == k:
            break
        m += lasit_newvec
    ########################################################################################### 
    print ('Iteration steps =', i+1)

    end = time.time()
    Eigenkets = np.dot(V[:,:m], sub_eigenket[:, :k])
    print ('Davidson time (no vind function):', round(end-start,4))
    # return (sub_eiegnvalue[:k], Eigenkets[:,:k])
    return (sub_eiegnvalue[:k]*27.21138624598853)

#print (davidson_A_matrix_hstack (A, 5))

In [33]:
# a = 1000 #dimension of H
# b = 3
# sparsity = 0.01
# H = np.zeros((a,a))
# for i in range(0,a):
#     H[i,i] = i + 1
# H = H + sparsity*np.random.randn(a,a)
# H = (H.T + H)/2   # A is a random Hermition

# E,vec = np.linalg.eig(H)
# idx = E.argsort()
# e = E[idx]
# print (' numpy   =', e[:10])  
# # Standard eigenvalues


# print (davidson_A_matrix (H, 10))

In [34]:
# print (27.21138624598853 * sTDA_init_p_davidson(5,1e-11))
# print (27.21138624598853 * sTDA_p_davidson(5,1e-11))
# print (27.21138624598853 * diagA_davidson(5,1e-11))


# Acetaldehyde Excited State energies (eV)
# [ 4.74398985  9.54998202  9.64102696 11.08547092 11.11130292] built-in method 
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302] reproduced A matrix from vind() function 




# 1e-4， Acetaldehyde
# Iteration = 9
# sTDA as initial guess and preconditioner Davidson time: 4.1452
# [ 4.7439899   9.54998214  9.6410271  11.08547116 11.11130306]
# Iteration = 10
# sTDA as preconditioner Davidson time: 4.9058
# [ 4.74398995  9.54998212  9.64102708 11.08547104 11.11130312]
# Iteration = 12
# A.diag as preconditioner Davidson time: 5.8675
# [ 4.7439899   9.54998214  9.64102705 11.08547104 11.1113031 ]
# !!!!!inaccurate!!!!



# 1e-5， Acetaldehyde
# Iteration = 10
# sTDA as initial guess and preconditioner Davidson time: 4.9217
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 12
# sTDA as preconditioner Davidson time: 6.4569
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 14
# A.diag as preconditioner Davidson time: 7.7608
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]


# 1e-6， Acetaldehyde
# Iteration = 11
# sTDA as initial guess and preconditioner Davidson time: 5.959
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 13
# sTDA as preconditioner Davidson time: 7.4774
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 16
# A.diag as preconditioner Davidson time: 9.8503
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]

# 1e-7， Acetaldehyde
# Iteration = 13
# sTDA as initial guess and preconditioner Davidson time: 7.7582
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 15
# sTDA as preconditioner Davidson time: 9.5674
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 17
# A.diag as preconditioner Davidson time: 10.9235
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]


# 1e-8， Acetaldehyde
# Iteration = 14
# sTDA as initial guess and preconditioner Davidson time: 8.9015
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# (600, 600)
# sTDA_A matrix built
# Iteration = 16
# sTDA as preconditioner Davidson time: 10.7825
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 18
# A.diag as preconditioner Davidson time: 12.3104
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]

# 1e-9， Acetaldehyde
# Iteration = 15
# sTDA as initial guess and preconditioner Davidson time: 10.7357
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 17
# sTDA as preconditioner Davidson time: 12.8471
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 19
# A.diag as preconditioner Davidson time: 14.5845
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]





# 1e-10， Acetaldehyd
# Iteration = 16
# sTDA as initial guess and preconditioner Davidson time: 12.1489
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 18
# sTDA as preconditioner Davidson time: 14.0931
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 20
# A.diag as preconditioner Davidson time: 15.9098
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]

# 1e-11， Acetaldehyd
# Iteration = 17
# sTDA as initial guess and preconditioner Davidson time: 13.3904
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 19
# sTDA as preconditioner Davidson time: 15.8702
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 21
# A.diag as preconditioner Davidson time: 17.3899
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]




# 1e-12， Acetaldehyde
# Iteration = 18
# sTDA as initial guess and preconditioner Davidson time: 13.9984
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 20
# sTDA as preconditioner Davidson time: 16.4304
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]
# Iteration = 22
# A.diag as preconditioner Davidson time: 18.2717
# [ 4.74398989  9.5499821   9.64102704 11.08547101 11.11130302]

In [35]:
#function to return distance between two atoms
# def distance (atom_A_id, atom_B_id):
# #   distance = R [atom_A_id, atom_B_id]
#     a = mol.atom_coord(atom_A_id) #coordinates of atom A
#     b = mol.atom_coord(atom_B_id) #coordinates of atom B
#     distance = np.linalg.norm (a - b)
#     return distance
# #(mol.atom_coord(atom_id)) is to return the coordinates of a certain atom, it is a ndarray


#array for the coordinates (in Bohr) of each atom 1 Bohr = 0.529177 Angstrom
#print (mol.unit)

In [36]:
#nocc = mol.nelectron 
# mol.nelectron is number of electrons